In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install glove-python-binary

In [ ]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/drive/MyDrive/FYP/IMDb/Glove/glove.model")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [ ]:
import tensorflow as tf
maxlen=66
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
d={}
for word,i in tokenizer.word_index.items():
  d[i]=word

In [ ]:
for i in X:
  for j in range(len(i)):
    if i[j]==0:
      pass
    else:
      p=d[i[j]]
      if p in glove_model.dictionary:
        i[j]=sum(glove_model.word_vectors[glove_model.dictionary[p]])
      else:
        i[j]=0
  print(i)


Streaming output truncated to the last 5000 lines.
 -3  2  1  2 -1  0  0  0 -3  0 -3  1 -2  1  1 -1  1 -2 -1  1  2 -2  0  0
  1  1  0 -1  0 -1 -3  0  2 -3  0  1  3  0  0 -1  0  0]
[ 0  0  0  0  0  2  4  1  2 -3  0 -2  0 -1  1 -1  0  3 -2  1 -1  0  0  0
  0  4 -1 -1 -2  0  0  0  1  1  1 -2  1  0  3  0 -2  0  1  0  1  2  1  2
  0  0 -1  4  0 -1  0  0  1  0  1  0  1  0 -6  0  3  1]
[ 1  1 -3  1  1  3  2  2  1  4  0  0 -3  0  0 -1  2  0  2 -3  2  4  1 -1
 -3  0  1  0 -1 -6  0  0  1  0  1  0  1  3  2  0  2  0 -1  0 -1  0 -1  0
  0 -2 -3  2  3 -1 -1 -1  0  1  1  0  0  0  0  0  2  0]
[ 0  0  2  1  0  1 -1 -3  2  0  3  0  0  1  1  1  3  0  0 -1 -1  0 -3  0
  0 -1  1  2 -1  0  0  1 -1  0  0 -2  1  0  0  0  0 -3  3  1  1  1  0 -6
 -1  0 -1  0  0  0  0  0 -1 -2 -1 -3  1  1  1  1  1  0]
[ 1  0  0  1  0 -1 -3  0  0  2  0 -1  0  1  1  0  3 -2  1  0  3  3  0  0
  0  1  1  0  0  1  0  0 -1  0  1  0  0  1  0  0  0  2  0  0  1  0  0  3
 -1  0  2  1  0  0  4  1 -3  0  0  2  0 -1 -2 -1  0 -1]
[-1  0  0  1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
display_metrics(y_test,y_pred)

0.514
0.5139213532526821
0.5143232661681855
0.5143031477228712


In [ ]:
from sklearn import svm 
clf = svm.SVC(kernel='linear',verbose=1,max_iter=10000)
clf.fit(X_train, y_train)

[LibSVM]

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=10000, verbose=1)

In [ ]:
y_pred=clf.predict(X_test)
display_metrics(y_test,y_pred)

0.50984
0.4914183052050497
0.508218176333014
0.5071203221418549


In [ ]:
from sklearn import tree
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred=DTclf.predict(X_test)
display_metrics(y_test,y_pred)

0.51712
0.5170769649898274
0.5170905134482151
0.5170940248632587
